In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain_openai import OpenAI
from langchain import  LLMChain
from langchain.tools import DuckDuckGoSearchRun 

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

In [3]:
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [4]:
obj = search.run("How can women treat cervical cancer?")
obj

"Typical cervical cancer symptoms are: unusual bleeding, like in between periods, after sex, or after menopause. vaginal discharge that looks or smells different than usual. pain in the pelvis ... Cervical cancer develops in a woman's cervix (the entrance to the uterus from the vagina). Its primary cause is the infection with high-risk human papillomaviruses (HPV). ... With a comprehensive approach to prevent, screen and treat, cervical cancer can be eliminated as a public health problem within a generation. A recent clinical trial has shown the effects of one such drug called tisotumab vedotin-tftv in helping women with recurrent cancers (cancer that has come back despite treatment). Kinne says the tisotumab vedotin-tftv attaches to cervical cancer cells and, once inside the cell, releases chemotherapy's toxic effects to kill the cancerous cell. Concurrent, cisplatin-based chemotherapy with radiation therapy is the standard of care for women who require radiation therapy for treatment

In [5]:
search.run("site:webmd.com How can I treat a sprained ankle?")

'1 /19. Deep vein thrombosis is a blood clot that forms inside a vein, usually deep within your leg. As many as 900,000 Americans a year get one, and up to 100,000 die because of it. The danger is ... 11 /14. This stands for "rest," "ice," "compression," and "elevation.". It\'s most useful for sprains and broken bones, but it can help if your feet are swollen for other reasons ... All high heels boost the risk of an ankle sprain.The most common, a lateral sprain, happens when you roll onto the outside of the foot. This stretches the ankle ligaments beyond their normal length. Monitor and Treat Your Symptoms. The symptoms of a cold, the flu, and COVID-19 can look and feel similar. They include a stuffy nose, tiredness, and muscle aches. No matter which virus causes your symptoms, if they\'re mild, the treatments are the same. Here are some things you can do: Drink fluids. Sleep more. Wound Care Step 3: Wash Around It. 8 /13. Put a little bit of soap on a washcloth and wash around the in

In [6]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

In [7]:
duck_wrapper("How to cure brest cencer?")

"An early-stage HER2-positive (HER2+) breast cancer is always a shock, but today's treatment options should give you hope. Over the past 2 decades, advances in medicine have brought us new ... Yoga. This form of exercise helps connect breath to movement. It slows your heart rate, blood pressure, and brain waves. Women who take yoga classes while having radiation for breast cancer say ... Early-stage breast cancer hasn't spread past your breast or auxiliary lymph nodes -- the ones near your armpit. Most people diagnosed with breast cancer in the U.S. are still in the early stages ... The goal is to kill as many cancer cells as possible without harming healthy ones. Most people get 10-20 minutes of radiation therapy 5 days a week for 3-7 weeks. The procedure itself is painless ... Mammograms. 4 /26. The earlier you find the disease, the easier it is to treat. Mammograms, an X-ray of the breast, can show tumors before they get large enough to feel. The American Cancer ..."

In [8]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [9]:
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [10]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [11]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [12]:
output_parser = CustomOutputParser()

In [13]:
llm = OpenAI(temperature=0.6)

In [14]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

c:\Users\rajan\Desktop\Coding\Jupyter Notebook\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.agent.LLMSingleActionAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [15]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, 
                                                    tools=tools, 
                                                    verbose=True)

In [16]:
agent_executor.run("How to Treat Cervical Cancer?")

c:\Users\rajan\Desktop\Coding\Jupyter Notebook\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: It is important to have a thorough understanding of the disease before determining treatment options.
Action: Search WebMD
Action Input: "Cervical Cancer Treatment"

Observation:June 15, 2023 -- A simple hysterectomy is a safe, effective treatment option for women with low-risk, early-stage cervical cancer, according to new research. As compared to the more complex ... "Patients with cervical cancer have few treatment options once their cancer comes back or spreads after initial treatment," said Ignace B. Vergote, MD, co-founder of European Network of ... Get the latest news on Cervical Cancer management, research, treatment advancements and more from WebMD. ... 2023 — One way of increasing the uptake of screening for cervical cancer is mailing ... External Beam Radiation. This treatment attacks cancer cells with high-energy particles (proton or particle therapy) or waves (X-rays). It kills or damages cells in one specific area instead o

'The treatment for cervical cancer depends on the stage and severity of the disease, and it is important to consult with a multidisciplinary team of medical professionals to determine the best course of action. This may include surgery, radiation therapy, chemotherapy, or targeted therapy. It is also important to be aware of alternative treatments, but always consult with a trusted medical professional before making any treatment decisions.'